## Start

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
# ! pip install transformers==2.2.2 -q
! pip install transformers -q
! pip install dgl-cu100 -q
! pip install word2vec -q

     |████████████████████████████████| 573kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 42.8MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
     |████████████████████████████████| 1.0MB 36.7MB/s 
     |████████████████████████████████| 16.8MB 188kB/s 
     |████████████████████████████████| 71kB 2.3MB/s 


In [0]:
import csv
import re
import pandas as pd
from pathlib import Path
import matplotlib.cm as cm
from fastai import *
from fastai.text import *
from fastai.callbacks import *
from fastai.metrics import *
import numpy as np
import pandas as pd

from pathlib import Path
from typing import *

import torch
import torch.optim as optim
import dgl
import word2vec

DGL backend not selected or invalid.  Assuming PyTorch for now.
Using backend: pytorch


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


In [0]:
import warnings
warnings.filterwarnings('ignore')

In [0]:
seed = 42

# python RNG
import random
random.seed(seed)

# pytorch RNGs
import torch
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

# numpy RNG
import numpy as np
np.random.seed(seed)

In [0]:
import os
os.chdir('/gdrive/My Drive/DEFINITION EXTRACTION/Definition')
! ls

 AutoTransfomer_attention.ipynb
 AutoTransfomer_attention_kfold_data_cased_symbols.ipynb
 AutoTransfomer_attention_kfold_data_cased_symbols_no_tuning.ipynb
 AutoTransfomer_attention_kfold_data_uncased_symbols.ipynb
 AutoTransfomer_attention_kfold_data_uncased_symbols_no_tuning.ipynb
 AutoTransfomer_attention_kfold.ipynb
 AutoTransfomer_attention_kfold_no_tuning.ipynb
 AutoTransfomer.ipynb
 bert_cached_lm_510_deft.test.raw
 bert_cached_lm_510_deft.train.raw
 bert-fastai.ipynb
 bert_ft
 BertTransfomer_attention.ipynb
 bertviz
 cnn_lstm_series_dep.ipynb
'Copy of AutoTransfomer_attention_kfold_data_cased_symbols.ipynb'
'Copy of AutoTransfomer_attention_kfold_data_uncased_symbols.ipynb'
'Copy of GCN_BERT_attention_kfold.ipynb'
'Copy of W00.csv'
 data_processing.ipynb
 deft.test.raw
 deft.train.raw
 End2End.ipynb
 lstm_attn.ipynb
 models
 runs
 train_lm.py
 VisualizeTransfomer.ipynb
 W00_BERT.ipynb
 WCL_cased_symbols.csv
 WCL_data1.csv
 WCL_data1.gsheet
 WCL_data2.csv
 WCL_data2.gsheet
 WCL_

In [0]:
model_name="bert-base-cased"
# model_name="roberta-base"
# model_name="xlnet-base-cased"

In [0]:
TEXT_COL = 'Psentence'
Y_COL = 'label'

In [0]:
import re
def multi_replace(chars_to_replace, char_to_place, from_string):
    for char in chars_to_replace:
        from_string = from_string.replace(char,char_to_place)
    return from_string

def preprocessing(sentence):
    remove_punct = multi_replace(',“”’.-{}[]()=+_!@#$%^&*<>?/|\\~`:;"','',sentence)
    return remove_punct

def load_data(file):
    df = pd.read_csv(file)
    # removing initial numbers
    df[TEXT_COL] = df[TEXT_COL].apply(lambda x : re.findall('^\s*\d*\s*\.?\s*(.*)',x)[0])
    # preprocessing
    # df[TEXT_COL] = df[TEXT_COL].apply(preprocessing)
    return df


In [0]:
merged = load_data('WCL_cased_symbols.csv')
merged.head(2)

,label,Psentence,Length,Dep1,Dep2,DepLabel
0,1,A Pianist is a person who plays the piano.,9,Pianist is is is person person plays plays piano,A Pianist person . a plays who piano the,nsubj ROOT ROOT ROOT attr attr relcl relcl dobj
1,0,Pingala has a sunlike nature and male energy.,8,has has has nature nature nature nature energy,Pingala nature . a sunlike and energy male,ROOT ROOT ROOT dobj dobj dobj dobj conj


In [0]:
def seq_len(row):
  return len(row[TEXT_COL].split())

merged['Length'] = merged.apply(seq_len, axis=1)

In [0]:
merged = merged[merged['Length']>4]
merged.shape

(4679, 6)

In [0]:
merged.head(3)

,label,Psentence,Length,Dep1,Dep2,DepLabel
0,1,A Pianist is a person who plays the piano.,9,Pianist is is is person person plays plays piano,A Pianist person . a plays who piano the,nsubj ROOT ROOT ROOT attr attr relcl relcl dobj
1,0,Pingala has a sunlike nature and male energy.,8,has has has nature nature nature nature energy,Pingala nature . a sunlike and energy male,ROOT ROOT ROOT dobj dobj dobj dobj conj
2,0,It's center is the city of Qorveh which is loc...,16,'s 's is is city city city of located located ...,It center city . the of located Qorveh which i...,ROOT ROOT ROOT ROOT attr attr attr prep relcl ...


## GCN

In [0]:
from keras.preprocessing.sequence import pad_sequences

class DataHelper(object):
    def __init__(self, df, mode='train', vocab=None):
        self.mode = mode
        
        self.df = df
        if self.mode=='train':
          content, label = self.get_content()
          self.label = self.label_to_onehot(label)

        else:
          content = self.get_content()
        
        if vocab is None:
            self.vocab = []
            try:
                print(f'File found')
                self.get_vocab()
            except FileNotFoundError:
                self.build_vocab(content, min_count=5)
        else:
            self.vocab = vocab

        self.d = dict(zip(self.vocab, range(len(self.vocab))))
        self.rev_d = { v:k for k,v in self.d.items()}
        self.content = [list(map(lambda x: self.word2id(x), doc.split(' '))) for doc in content]
        self.lengths = [[len(s)] for s in self.content]
        self.content = pad_sequences(self.content, maxlen=64, dtype="long", truncating="post", padding="post")


    def label_to_onehot(self, label_str):
      return np.array(self.df[Y_COL])
        
    def get_content(self):
        if self.mode=='train':
          return list(self.df[TEXT_COL]), list(self.df[Y_COL])
        else:
          return list(self.df[TEXT_COL])

    def word2id(self, word):
        try:
            result = self.d[word]
        except KeyError:
            result = self.d['UNK']
        return result

    def get_vocab(self):
        with open('vocab-5.txt') as f:
            vocab = f.read()
            self.vocab = vocab.split('\n')

    def build_vocab(self, content, min_count=10):
        vocab = []
        for c in content:
            words = c.split(' ')
            for word in words:
                if word not in vocab:
                    vocab.append(word)
        freq = dict(zip(vocab, [0 for i in range(len(vocab))]))
        for c in content:
            words = c.split(' ')
            for word in words:
                freq[word] += 1
        results = []
        for word in freq.keys():
            if freq[word] < min_count:
                continue
            else:
                results.append(word)
        results.insert(0, 'UNK')
        with open('vocab.txt', 'w') as f:
            f.write('\n'.join(results))
        self.vocab = results

    def count_word_freq(self, content):
        freq = dict(zip(self.vocab, [0 for i in range(len(self.vocab))]))
        for c in content:
            words = c.split(' ')
            for word in words:
                freq[word] += 1
        with open('freq.csv', 'w') as f:
            writer = csv.writer(f)
            results = list(zip(freq.keys(), freq.values()))
            writer.writerows(results)

    def batch_iter(self, batch_size, num_epoch):
        for i in range(num_epoch):
            num_per_epoch = int(len(self.content) / batch_size)
            for batch_id in range(num_per_epoch):
                start = batch_id * batch_size
                end = min((batch_id + 1) * batch_size, len(self.content))
                content = self.content[start:end]
                label = self.label[start:end]
                yield (content), torch.LongTensor(label).cuda(), i


Using TensorFlow backend.


In [0]:
test_df = merged

In [0]:
train_data_helper = DataHelper(merged, mode='train')
test_data_helper = DataHelper(merged, mode='test')

File found
File found


## Transformer

In [0]:
 %tensorflow_version 1.x 
from transformers import BertTokenizer, GPT2Tokenizer
from transformers import BertForSequenceClassification,  AutoModelForSequenceClassification

from transformers import BertPreTrainedModel, BertModel

tokenizer = BertTokenizer.from_pretrained(model_name, output_attentions=True)

TensorFlow is already loaded. Please restart the runtime to change versions.


## Preprocessing

In [0]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 64

def tokenise_pad(df):
  X = [tokenizer.encode(x, add_special_tokens=True) for x in df[TEXT_COL]]
  X = pad_sequences(X, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
  return X


X = tokenise_pad(merged)
X_test = tokenise_pad(test_df)

In [0]:
def return_attention_masks(X):
  attention_masks = []
  for seq in X:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)
  attention_masks = np.array(attention_masks)  
  return attention_masks

In [0]:
attention_masks = return_attention_masks(X)
test_attention_masks = return_attention_masks(X_test)

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [0]:
y = np.array(merged[Y_COL])

## Models

In [0]:
def cal_PMI(helper, window_size=20):
    content, _ = helper.get_content()
    pair_count_matrix = np.zeros((len(helper.vocab), len(helper.vocab)), dtype=int)
    word_count =np.zeros(len(helper.vocab), dtype=int)

    for sentence in content:
        sentence = sentence.split(' ')
        for i, word in enumerate(sentence):
            try:
                word_count[helper.d[word]] += 1
            except KeyError:
                continue
            start_index = max(0, i - window_size)
            end_index = min(len(sentence), i + window_size)
            for j in range(start_index, end_index):
                if i == j:
                    continue
                else:
                    target_word = sentence[j]
                    try:
                        pair_count_matrix[helper.d[word], helper.d[target_word]] += 1
                    except KeyError:
                        continue

    total_count = np.sum(word_count)
    word_count = word_count / total_count
    pair_count_matrix = pair_count_matrix / total_count

    pmi_matrix = np.zeros((len(helper.vocab), len(helper.vocab)), dtype=float)
    for i in range(len(helper.vocab)):
        for j in range(len(helper.vocab)):
            pmi_matrix[i, j] = np.log(
                pair_count_matrix[i, j] / (word_count[i] * word_count[j])
            )

    pmi_matrix = np.nan_to_num(pmi_matrix)

    pmi_matrix = np.maximum(pmi_matrix, 0.0)

    edges_weights = [0.0]
    count = 1
    edges_mappings = np.zeros((len(helper.vocab), len(helper.vocab)), dtype=int)
    for i in range(len(helper.vocab)):
        for j in range(len(helper.vocab)):
            if pmi_matrix[i, j] != 0:
                edges_weights.append(pmi_matrix[i, j])
                edges_mappings[i, j] = count
                count += 1

    edges_weights = np.array(edges_weights)

    edges_weights = edges_weights.reshape(-1, 1)
    # print(edges_weights.shape)
    edges_weights = torch.Tensor(edges_weights)

    return edges_weights, edges_mappings, count


In [0]:
def gcn_msg(edge):
    return {'m': edge.src['h'], 'w': edge.data['w']}


def gcn_reduce(node):
    w = node.mailbox['w']

    new_hidden = torch.mul(w, node.mailbox['m'])

    new_hidden,_ = torch.max(new_hidden, 1)

    node_eta = torch.sigmoid(node.data['eta'])
    # node_eta = F.leaky_relu(node.data['eta'])

    # new_hidden = node_eta * node.data['h'] + (1 - node_eta) * new_hidden
    # print(new_hidden.shape)

    return {'h': new_hidden}


class GCNModel(torch.nn.Module):
    def __init__(self,
                 hidden_size_node,
                 vocab,
                 n_gram,
                 edges_num,
                 edges_matrix,
                 max_length=350,
                 trainable_edges=True,
                 pmi=None,
                 cuda=True
                 ):
        super(GCNModel, self).__init__()

        self.is_cuda = cuda
        self.vocab = vocab
        
        self.seq_edge_w = torch.nn.Embedding(edges_num, 1)
        print(edges_num)
        print(pmi.shape)

        self.node_hidden = torch.nn.Embedding(len(vocab), hidden_size_node)
        
        self.seq_edge_w = torch.nn.Embedding.from_pretrained(pmi, freeze=True)
            
        self.edges_num = edges_num
        if trainable_edges:
            self.seq_edge_w = torch.nn.Embedding.from_pretrained(torch.ones(edges_num, 1), freeze=False)
        else:
            self.seq_edge_w = torch.nn.Embedding.from_pretrained(pmi, freeze=True)

        self.hidden_size_node = hidden_size_node

        self.node_hidden.weight.data.copy_(torch.tensor(self.load_word2vec('glove.6B.200d.vec.txt')))
        self.node_hidden.weight.requires_grad = True

        self.len_vocab = len(vocab)

        self.ngram = n_gram
        self.d = dict(zip(self.vocab, range(len(self.vocab))))
        self.max_length = max_length
        self.edges_matrix = edges_matrix

        self.dropout = torch.nn.Dropout(0.5)
        self.activation = torch.nn.ReLU()

        

    def word2id(self, word):
        try:
            result = self.d[word]
        except KeyError:
            result = self.d['UNK']

        return result

    def load_word2vec(self, word2vec_file):
        model = word2vec.load(word2vec_file)
        embedding_matrix = []
        for word in self.vocab:
            try:
                embedding_matrix.append(model[word])
            except KeyError:
                # print(word)
                embedding_matrix.append(model['the'])
        embedding_matrix = np.array(embedding_matrix)
        return embedding_matrix

    

    def add_seq_edges(self, doc_ids: list, old_to_new: dict):
        edges = []
        old_edge_id = []
        for index, src_word_old in enumerate(doc_ids):
            src = old_to_new[int(src_word_old.item())]
            for i in range(max(0, index - self.ngram), min(index + self.ngram + 1, len(doc_ids))):
                dst_word_old = doc_ids[i].item()
                dst = old_to_new[dst_word_old]

                # - first connect the new sub_graph
                edges.append([src, dst])
                # - then get the hidden from parent_graph
                old_edge_id.append(self.edges_matrix[src_word_old, dst_word_old])

            # self circle
            edges.append([src, src])
            old_edge_id.append(self.edges_matrix[src_word_old, src_word_old])
        return edges, old_edge_id

    def seq_to_graph(self, doc_ids: list, doc_length) -> dgl.DGLGraph():
        doc_ids = doc_ids[0:doc_length]
        if len(doc_ids) > self.max_length:
            doc_ids = doc_ids[:self.max_length]
        local_vocab = set(doc_ids)

        old_to_new = {}
        for i,j in enumerate(local_vocab):
          old_to_new[j.item()] = i
        # old_to_new = dict(zip(local_vocab, range(len(local_vocab))))

        # print(old_to_new)

        if self.is_cuda:
            local_vocab = torch.tensor(list(local_vocab)).cuda()
        else:
            local_vocab = torch.tensor(list(local_vocab))

        sub_graph = dgl.DGLGraph()

        sub_graph.add_nodes(len(local_vocab))
        local_node_hidden = self.node_hidden(local_vocab)

        sub_graph.ndata['h'] = local_node_hidden

        seq_edges, seq_old_edges_id = self.add_seq_edges(doc_ids, old_to_new)

        edges, old_edge_id = [], []
        
        edges.extend(seq_edges)

        old_edge_id.extend(seq_old_edges_id)

        if self.is_cuda:
            old_edge_id = torch.LongTensor(old_edge_id).cuda()
        else:
            old_edge_id = torch.LongTensor(old_edge_id)

        srcs, dsts = zip(*edges)
        sub_graph.add_edges(srcs, dsts)
        try:
            seq_edges_w = self.seq_edge_w(old_edge_id)
        except RuntimeError:
            print(old_edge_id)
        sub_graph.edata['w'] = seq_edges_w

        return sub_graph

    def forward(self, doc_ids, doc_lengths):
        sub_graphs = [self.seq_to_graph(doc, length) for doc, length in zip(doc_ids, doc_lengths)]

        batch_graph = dgl.batch(sub_graphs)

        batch_graph.update_all(
            message_func=dgl.function.src_mul_edge('h', 'w', 'weighted_message'),
            reduce_func=dgl.function.max('weighted_message', 'h')
        )

        h1 = dgl.sum_nodes(batch_graph, feat='h')
        drop1 = self.dropout(h1)
        act1 = self.activation(drop1)
        # l = self.Linear(act1)
        return act1


In [0]:
class BertFeature(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.bert = BertModel(config)
        self.init_weights()
        self.features = config.hidden_size

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):

            bert_outputs = self.bert(
              input_ids,
              attention_mask=attention_mask,
              token_type_ids=token_type_ids,
              position_ids=position_ids,
              head_mask=head_mask,
              inputs_embeds=inputs_embeds,
            )
            pooled_output = bert_outputs[1]

            return pooled_output

## Main Model

In [0]:
class BertGCN(nn.Module):
  def __init__(self, hidden, vocabulary, n_grams, edges_mappings, edges_weights, count, num_classes, dropout_prob):
    super(BertGCN, self).__init__()
    self.bert = BertFeature.from_pretrained('bert-base-cased', output_attentions=False)
    self.gcn = GCNModel(hidden_size_node = hidden,
                            vocab = vocabulary,
                            n_gram = n_grams,
                            edges_matrix = edges_mappings,
                            edges_num = count,
                            trainable_edges = True, 
                            pmi = edges_weights, 
                            cuda = True
                            )
    self.num_classes = num_classes
    self.dropout = nn.Dropout(dropout_prob)
    self.bn1 = nn.BatchNorm1d(self.bert.features)
    self.relu1 = torch.nn.ReLU()
    self.relu2 = torch.nn.ReLU()
    
    self.bn2 = nn.BatchNorm1d(hidden)
    
    self.classifier = nn.Linear(self.bert.features + hidden, self.num_classes)

  def forward(self, input_ids, attention_mask, input_gcn, sent_len, labels=None):
    bert_out = self.bert(input_ids, attention_mask)
    gcn_out = self.gcn(input_gcn, sent_len)

    # return bert_out, gcn_out

    # bert_out = self.bn1(bert_out)
    # gcn_out = self.bn2(gcn_out)
    
    # bert_out = self.relu1(bert_out)
    # gcn_out = self.relu2(gcn_out)

    out = torch.cat((bert_out, gcn_out), 1)
    # pooled_output = self.dropout(out)
    logits = self.classifier(out)

    outputs = (logits,)

    if labels is not None:
        if self.num_classes == 1:
            loss_fct = MSELoss()
            loss = loss_fct(logits.view(-1), labels.view(-1))
        else:
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_classes), labels.view(-1))
        outputs = (loss,) + outputs
    return outputs

## DEBUG

In [0]:
edges_weights, edges_mappings, count = cal_PMI(train_data_helper, 5)

In [0]:
# model = BertGCN(hidden = 200,
#                 vocabulary = train_data_helper.vocab,
#                 n_grams = 5,
#                 edges_mappings = edges_mappings,
#                 edges_weights = edges_weights, 
#                 count = count, 
#                 num_classes = 2, 
#                 dropout_prob = 0.3
#                 )

# Training

In [0]:
device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")
device

device(type='cuda', index=0)

In [0]:
from sklearn.utils.extmath import softmax
from sklearn.metrics import classification_report, f1_score
BATCH_SIZE = 32


def flat_accuracy(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def flat_f1(preds, labels):
    preds = softmax(preds)
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, pred_flat)


In [0]:
X_gcn = train_data_helper.content
X_len = train_data_helper.lengths
X_test_gcn = test_data_helper.content
X_test_len = test_data_helper.lengths

In [0]:
X_gcn = np.array(X_gcn)
X_test_gcn = np.array(X_test_gcn)

X_len = np.array(X_len)
X_test_len = np.array(X_test_len)

In [0]:
X_test.shape, X_test_gcn.shape, test_attention_masks.shape, X_test_len.shape

((4679, 64), (4679, 64), (4679, 64), (4679, 1))

In [0]:
X_test = torch.tensor(X_test)
X_test_masks = torch.tensor(test_attention_masks)
X_test_gcn = torch.tensor(X_test_gcn)
X_test_len = torch.tensor(X_test_len)

test_data = TensorDataset(X_test, X_test_masks, X_test_gcn, X_test_len)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=BATCH_SIZE)

### Loop

In [0]:
from sklearn.model_selection import KFold,StratifiedKFold
from tqdm import tqdm
import sys

auto_model = None

results = pd.DataFrame()

S = []

nfolds = 10

kf = StratifiedKFold(nfolds, shuffle=True, random_state=2018)
for f,(tr, val) in enumerate(kf.split(X, y)):
    print(f'--------------------------------------------------------------------------------Fold # {f}--------------------------------------------------------------------')

    X_train, X_val = X[tr], X[val]
    y_train, y_val = y[tr], y[val]
    X_train_gcn, X_val_gcn = X_gcn[tr], X_gcn[val]
    X_train_masks, X_val_masks = attention_masks[tr], attention_masks[val]
    X_train_len, X_val_len = X_len[tr], X_len[val]
                                         
    # Convert all of our data into torch tensors, the required datatype for our model
    X_train = torch.tensor(X_train)
    X_val = torch.tensor(X_val)

    y_train = torch.tensor(y_train)
    y_val = torch.tensor(y_val)

    X_train_masks = torch.tensor(X_train_masks)
    X_val_masks = torch.tensor(X_val_masks)

    X_train_gcn = torch.tensor(X_train_gcn)
    X_val_gcn = torch.tensor(X_val_gcn)

    X_train_len = torch.tensor(X_train_len)
    X_val_len = torch.tensor(X_val_len)

    
    # Create an iterator of data with torch DataLoader 
    train_data = TensorDataset(X_train, X_train_masks, X_train_gcn, X_train_len, y_train)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

    validation_data = TensorDataset(X_val, X_val_masks, X_val_gcn, X_val_len, y_val)
    validation_sampler = SequentialSampler(validation_data)
    validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=BATCH_SIZE)



    ## Model
    del auto_model
    auto_model = BertGCN(hidden = 200,
                vocabulary = train_data_helper.vocab,
                n_grams = 5,
                edges_mappings = edges_mappings,
                edges_weights = edges_weights, 
                count = count, 
                num_classes = 2, 
                dropout_prob = 0.3
                )


    auto_model.to(device)

    # Optimizer
    num_total_steps = 1000
    num_warmup_steps = 100
    lr = 3e-6
    param_optimizer = list(auto_model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
        'weight_decay_rate': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=lr)
    
    best_loss = None

    auto_model.train()  
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    n_epochs = 4

    for epoch in (range(n_epochs)):  
      # Training Loop
      auto_model.train()
      for step, batch in (enumerate(train_dataloader)):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_input_gcn ,b_len, b_labels = batch

        optimizer.zero_grad()
        outputs = auto_model(b_input_ids, 
                             b_input_mask, 
                             b_input_gcn, 
                             b_len, 
                             b_labels)

        loss, logits = outputs[:2]
        # print(loss)
        # print(logits)
        # sys.exit()

        loss.backward()
        optimizer.step()
        
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1


      # Validation Loop
          
      auto_model.eval()
      eval_loss, eval_accuracy = 0, 0
      nb_eval_steps, nb_eval_examples = 0, 0

      yt, yp = [], []
      for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_input_gcn ,b_len, b_labels = batch

        with torch.no_grad():
          outputs = auto_model(b_input_ids, 
                                attention_mask=b_input_mask,
                                input_gcn = b_input_gcn,
                                sent_len = b_len,
                                labels= b_labels)        
          loss, logits = outputs[:2]
        logits = logits.detach().cpu().numpy()
        
        preds = softmax(logits)
        pred_ids = np.argmax(preds, axis=1).flatten()

        label_ids = b_labels.to('cpu').numpy()

        eval_loss += loss.item()
        eval_accuracy += flat_accuracy(logits, label_ids)

        yt = yt + label_ids.tolist()
        yp = yp + pred_ids.tolist()

        nb_eval_steps +=1
        
      print("Epoch {} | Train loss: {} | Validation Loss: {} ".format(epoch,
                                                                      tr_loss/nb_tr_steps,
                                                                      eval_loss/nb_eval_steps, 
                                                                    ))
      if best_loss == None or best_loss > eval_loss/nb_eval_steps:
        best_loss = eval_loss/nb_eval_steps
        torch.save(auto_model.state_dict(), 'models/check_bertgcn.pth')
        print(f'Saving model')
      
    
    # test predictions
    test_preds = []
    auto_model.load_state_dict(torch.load('models/check_bertgcn.pth'))
    auto_model.eval()
    print('Loading best model')
    
    for batch in test_dataloader:
      batch = tuple(t.to(device) for t in batch)
      b_input_ids, b_input_mask, b_input_gcn ,b_len = batch

      with torch.no_grad():
        outputs = auto_model(b_input_ids, 
                              b_input_mask,
                              b_input_gcn,
                              b_len
                            )        
        logits = outputs[0]
      logits = logits.detach().cpu().numpy()
      preds = softmax(logits)[:, 1]        
      test_preds = test_preds + preds.tolist()
    
    results[f'Fold_{f}'] = test_preds
    


    #if epoch == n_epochs:
    print(classification_report(yt, yp))   
    s = classification_report(yt, yp, output_dict=True)
    S.append(s['1']['f1-score'])


--------------------------------------------------------------------------------Fold # 0--------------------------------------------------------------------
135768
torch.Size([135768, 1])
Epoch 0 | Train loss: 0.4182675943681688 | Validation Loss: 0.2164306362469991 
Saving model
Epoch 1 | Train loss: 0.28425214130600746 | Validation Loss: 0.14387961328029633 
Saving model
Epoch 2 | Train loss: 0.21937864114803196 | Validation Loss: 0.14172490065296492 
Saving model
Epoch 3 | Train loss: 0.18122795619296306 | Validation Loss: 0.13333603392044704 
Saving model
Loading best model
              precision    recall  f1-score   support

           0       0.98      0.96      0.97       280
           1       0.94      0.97      0.96       188

    accuracy                           0.96       468
   macro avg       0.96      0.97      0.96       468
weighted avg       0.96      0.96      0.96       468

--------------------------------------------------------------------------------Fold # 1

In [0]:
S = np.array(S)
S.mean()

0.9704762728792533

In [0]:
results.head()

,Fold_0,Fold_1,Fold_2,Fold_3,Fold_4
0,0.473291,0.582156,0.773717,0.515028,0.552867
1,0.980621,0.788923,0.934692,0.872770,0.941588
2,0.576671,0.356888,0.449433,0.471735,0.473566
3,0.099559,0.111903,0.279063,0.209035,0.186653
4,0.988700,0.802909,0.962213,0.950028,0.977429


In [0]:
results['Avg'] = 0
for i in range(nfolds):
  results['Avg'] += results[f'Fold_{i}']
results['Avg'] /=nfolds

In [0]:
test_df['Logits'] = results['Avg']
test_df['PredictedClass'] = (test_df['Logits']>=0.5)*1
test_df.head()

,Subject,Sentence,Label\n,Logits,PredictedClass
0,biology,Living things are highly organized and structu...,NaN,0.579412,1
1,biology,The atom is the smallest and most fundamental ...,NaN,0.903719,1
2,biology,It consists of a nucleus surrounded by electro...,NaN,0.465659,0
3,biology,"At its most fundamental level , life is made u...",NaN,0.177243,0
4,biology,Matter is any substance that occupies space an...,NaN,0.936256,1


In [0]:
test_df.to_csv('sub/submission_hybrid_bert.csv', index=False)

In [0]:
# torch.save(auto_model.state_dict(), 'models/bert_attn_97_10fold.pth')

In [0]:
# auto_model.load_state_dict(torch.load('models/bert_attn_97_10fold.pth'))
# print(f'Done')

In [0]:
test_df.shape

(859, 5)